In [1]:
#Libraries

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import re
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
from functools import partial
import torch
import random
from sklearn.model_selection import train_test_split
#!pip install transformers
#from transformers import BertTokenizer, BertModel
#import spacy

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')
random.seed(10)

Thu Apr 29 15:05:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

True
Using device: cuda


In [ ]:
#data = pd.read_csv("data.csv")
data = pd.read_csv("data.csv")
data.head()
print(len(data))

4155


In [20]:
text = []
for i in range(len(data)):
  t = data.loc[i][6]


  text.append((t, data.loc[i][5]))


In [21]:
pad_word = "<pad>"
bos_word = "<s>"
eos_word = "</s>"
unk_word = "<unk>"
pad_id = 0
bos_id = 1
eos_id = 2
unk_id = 3
    
def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class Vocabulary:
    def __init__(self):
        self.word_to_id = {pad_word: pad_id, bos_word: bos_id, eos_word:eos_id, unk_word: unk_id}
        self.word_count = {}
        self.id_to_word = {pad_id: pad_word, bos_id: bos_word, eos_id: eos_word, unk_id: unk_word}
        self.num_words = 4
    
    def get_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [bos_id] + [self.word_to_id[word] if word in self.word_to_id \
                               else unk_id for word in sentence.split()] + \
                               [eos_id]
        return sent_ids
    
    def tokenized_sentence(self, sentence):
        sent_ids = self.get_ids_from_sentence(sentence)
        return [self.id_to_word[word_id] for word_id in sent_ids]

    def decode_sentence_from_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [bos_id, eos_id, pad_id]:
                # Skip these words
                continue
            else:
                words.append(self.id_to_word[word_id])
        return ' '.join(words)

    def add_words_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in sentence.split():
            if word not in self.word_to_id:
                # add this word to the vocabulary
                self.word_to_id[word] = self.num_words
                self.id_to_word[self.num_words] = word
                self.word_count[word] = 1
                self.num_words += 1
            else:
                # update the word count
                self.word_count[word] += 1

vocab = Vocabulary()
for src, tgt in text:
    vocab.add_words_from_sentence(src)
    vocab.add_words_from_sentence(tgt)
print(f"Total words in the vocabulary = {vocab.num_words}")

Total words in the vocabulary = 56347


In [62]:
print(vocab.id_to_word[1])

<s>


Seq2Seq

In [ ]:
def predict_greedy(model, sentence, max_length=100):
    """Make predictions for the given input using greedy inference.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: A input string.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        Model's predicted greedy response for the input, represented as string.
    """

    # You should make only one call to model.encode() at the start of the function, 
    # and make only one call to model.decode() per inference step.
    model.eval()
    source = torch.unsqueeze(torch.tensor(vocab.get_ids_from_sentence(sentence)).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    sent = [start]
    i = 0
    while start != eos_id and i < 100:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
          start = torch.argmax(out[0], 0)
          sent.append(start.item())
          i += 1
    sent = vocab.decode_sentence_from_ids(sent)
    
    return sent
print(predict_greedy(baseline_model, text[2][0]))
print(text[2][1])


baltimore longitudinal study of aging
baltimore longitudinal study of aging blsa 


In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
score = 0
for j in range(0,100):

  if jaccard(text[j][1], predict_greedy(baseline_model, text[j][0])) >= 0.5:
    score += 1
print(score)


25


In [ ]:
def predict_beam(model, sentence, k=3, max_length=100, thresh=-9999):
    """Make predictions for the given inputs using beam search.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: An input sentence, represented as string.
        k: The size of the beam.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        A list of k beam predictions. Each element in the list should be a string
        corresponding to one of the top k predictions for the corresponding input,
        sorted in descending order by its final score.
    """

    # Implementation tip: once an eos_token has been generated for any beam, 
    # remove its subsequent predictions from that beam by adding a small negative 
    # number like -1e9 to the appropriate logits. This will ensure that the 
    # candidates are removed from the beam, as its probability will be very close
    # to 0. Using this method, uou will be able to reuse the beam of an already 
    # finished candidate

    # Implementation tip: while you are encouraged to keep your tensor dimensions
    # constant for simplicity (aside from the sequence length), some special care
    # will need to be taken on the first iteration to ensure that your beam
    # doesn't fill up with k identical copies of the same candidate.
    
    # You are welcome to tweak alpha
    alpha = 0.9
    model.eval()
    beams = []
    curr = []
    source = torch.unsqueeze(torch.tensor(vocab.get_ids_from_sentence(sentence)).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    
    out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
    out = torch.log_softmax(out[0], 0)
    values, start = torch.topk(out, k, 0)
    for i in range(len(values)):
      # Each beam contains the log probs at its first index and the hidden states at its last index
      beams.append([values[i], start[i].item(), hid])
    
    generation = []
    i = 0
    while i < k:
      curr = []
      for j in beams:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(j[-2]).cuda(), 0), j[-1], x, mask)
          out = torch.log_softmax(out[0], 0)
          values, start = torch.topk(out, k, 0)
          for z in range(len(values)):
            temp = j.copy()
            temp[0] = values[z] + temp[0]
            temp.insert(-1, start[z].item())
            temp[-1] = hid
            curr.append(temp)
      curr = sorted(curr,reverse=True, key=lambda x: x[0])
      curr = curr[0:k - i]
      beams = []
      for j in curr:
        if j[-2] == eos_id or len(j) > 20:
          generation.append(j[:-1])
          i +=1
        else:
          beams.append(j)
    final = []
    generation = sorted(generation, reverse=True, key=lambda x: x[0]/(len(x)-1)**alpha)
    for i in generation:
      if i[0].item() > thresh:
        final.append(vocab.decode_sentence_from_ids(i[1:]))
    return final

    
    # YOUR CODE HERE
    ...

In [ ]:
testing = {}
for i in range(0, len(test)):
  if test[i][0] not in testing.keys():
    predictions = predict_beam(baseline_model, test[i][0], thresh=-2)
    testing[test[i][0]] = (predictions, [test[i][1]])
  else:
    testing[test[i][0]][1].append(test[i][1])

In [ ]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  true_pred = i[1].copy()
  for j in predictions:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [ ]:
print(tp)
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))

In [ ]:
testing = {}
for i in range(0, len(text)):
  if text[i][0] not in testing.keys():
    predictions = predict_beam(baseline_model, text[i][0], thresh=-2)
    testing[text[i][0]] = (predictions, [text[i][1]])
  else:
    testing[text[i][0]][1].append(text[i][1])

In [ ]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  true_pred = i[1].copy()
  for j in predictions:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [ ]:
print(tp)
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))

3349
1967
806
0.7072114876992925


In [ ]:
score = 0

for j in range(0,len(test), 10):
  predictions = predict_beam(baseline_model, test[j][0])
  for i in predictions:
    if jaccard(test[j][1], i) >= 0.5:
      score += 1
      break
print(score)

101


In [ ]:
print(score*10/len(test))

0.8500400962309543


In [ ]:
print("testing_accuracy")
print(score/len(test))

testing_accuracy
0.863672814755413


In [ ]:
score = 0
for j in range(0,len(text)):
  predictions = predict_beam(baseline_model, text[j][0])
  for i in predictions:
    if jaccard(text[j][1], i) >= 0.5:
      score += 1
      break
print("training_accuracy")
print(score/len(text))

training_accuracy
0.8803301237964236


Tried testing on unfiltered dataset

In [ ]:
import math
test_unfilter = pd.read_csv("data_false.csv")
test_unfilter = test_unfilter[test_unfilter.text.notnull()].reset_index()
text_2 = []
for i in range(len(test_unfilter)):

  t = test_unfilter.loc[i][7]

  text_2.append((t, test_unfilter.loc[i][6]))


In [ ]:

print(text_2[0])

(' Federal Reserve Bank of Richmond S1. Accounting for low enrollment and graduation rates at four-year colleges when returns to graduation are high Several studies claimed that there is underinvestment in education. Judd (2000) combines Capital Asset Pricing Model (CAPM) techniques with the indivisibility of human capital to compare the return to four-year college graduation with assets of similar risk to find an excess of return to the college investment option. Heckman, Lochner, and Todd (2008) evaluate the internal rate of return of the four-year college investment option relative to work to find that since 1960, internal rates of return have been around 10 percent or higher, depending on the cohort and different specifications of labor markets and taxes. Cunha, Heckman, and Navarro (2005), using data from NLSY/1979, extend the analysis to evaluate the internal rate of return for the marginal student, the agent with the lowest observable measures of ability who enrolls in four-year

In [ ]:
score = 0
for j in range(0,len(text_2), 50):
  predictions = predict_beam(baseline_model, text_2[j][0])
  for i in predictions:
    if jaccard(text_2[j][1], i) >= 0.5:
      score += 1
      break
print("training_accuracy")
print(score/(len(text_2)/50))

training_accuracy
0.5281361002056459


In [3]:
import spacy
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 2.1MB 30.6MB/s 
     |████████████████████████████████| 1.2MB 56.8MB/s 
     |████████████████████████████████| 3.3MB 52.6MB/s 
     |████████████████████████████████| 901kB 55.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=af0fa2858ba7807229a204ab4f6459e53c22e51a370e95d84114e44a591783ba
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
sent = SentenceTransformer("facebook-dpr-ctx_encoder-single-nq-base")

In [7]:
sent_sep = spacy.load("en_core_web_sm")

In [10]:
data = pd.read_csv("data.csv")

In [95]:
sent_emb = []
for j in range(len(data)):
  x = sent_sep(data.loc[j][6])
  y = list(x.sents)
  z = []
  for i in y:
    z.append(i.text)
  y = sent.encode(z)
  sent_emb.append(y)

In [97]:
labels = []
for j in range(len(data)):
  labels.append(data.loc[j][5])

In [98]:
data["embeddings"] = sent_emb

In [102]:
data.to_csv("data_with_embeddings.csv")

In [103]:
sent_emb, test_x, labels, test_y = train_test_split(sent_emb, labels, test_size=0.3, random_state=10)

In [77]:
import torch.nn as nn
class Seq2seqBaseline(nn.Module):
    def __init__(self, vocab, emb_dim = 300, hidden_dim = 300, num_layers = 2, dropout=0.1):
        super().__init__()

        # Initialize your model's parameters here. To get started, we suggest
        # setting all embedding and hidden dimensions to 300, using encoder and
        # decoder GRUs with 2 layers, and using a dropout rate of 0.1.

        # Implementation tip: To create a bidirectional GRU, you don't need to
        # create two GRU networks. Instead use nn.GRU(..., bidirectional=True).
        
        self.num_words = vocab.num_words
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.E = nn.Embedding(self.num_words, emb_dim)

        self.encoder = nn.GRU(768, hidden_dim, 2, dropout=dropout, bidirectional=True)
        self.decoder = nn.GRU(768, hidden_dim, 2, dropout=dropout, bidirectional=False)
        self.lin = nn.Linear(300, vocab.num_words)
        # YOUR CODE HERE

    def encode(self, source):
        """Encode the source batch using a bidirectional GRU encoder.

        Args:
            source: An integer tensor with shape (max_src_sequence_length,
                batch_size) containing subword indices for the source sentences.

        Returns:
            A tuple with three elements:
                encoder_output: The output hidden representation of the encoder 
                    with shape (max_src_sequence_length, batch_size, hidden_size).
                    Can be obtained by adding the hidden representations of both 
                    directions of the encoder bidirectional GRU. 
                encoder_mask: A boolean tensor with shape (max_src_sequence_length,
                    batch_size) indicating which encoder outputs correspond to padding
                    tokens. Its elements should be True at positions corresponding to
                    padding tokens and False elsewhere.
                encoder_hidden: The final hidden states of the bidirectional GRU 
                    (after a suitable projection) that will be used to initialize 
                    the decoder. This should be a tensor h_n with shape 
                    (num_layers, batch_size, hidden_size). Note that the hidden 
                    state returned by the bi-GRU cannot be used directly. Its 
                    initial dimension is twice the required size because it 
                    contains state from two directions.

        The first two return values are not required for the baseline model and will
        only be used later in the attention model. If desired, they can be replaced
        with None for the initial implementation.
        """

        # Implementation tip: consider using packed sequences to more easily work
        # with the variable-length sequences represented by the source tensor.
        # See https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.PackedSequence.

        # https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

        # Implementation tip: there are many simple ways to combine the forward
        # and backward portions of the final hidden state, e.g. addition, averaging,
        # or a linear transformation of the appropriate size. Any of these
        # should let you reach the required performance.

        # Compute a tensor containing the length of each source sequence.

        hid = torch.zeros(4, source.shape[1], 300).cuda()
        encoder_mask = source != 0
        x, hid = self.encoder(source, hid)

        x = x[:,:,:self.hidden_dim] + x [:,:,self.hidden_dim:]
        hid = hid[:self.num_layers,:,:] + hid[self.num_layers:,:,:]

        # YOUR CODE HERE
        ...
        return  x, encoder_mask, hid

    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):
        """Run the decoder GRU for one decoding step from the last hidden state.

        The third and fourth arguments are not used in the baseline model, but are
        included for compatibility with the attention model in the next section.

        Args:
            decoder_input: An integer tensor with shape (1, batch_size) containing 
                the subword indices for the current decoder input.
            last_hidden: A pair of tensors h_{t-1} representing the last hidden
                state of the decoder, each with shape (num_layers, batch_size,
                hidden_size). For the first decoding step the last_hidden will be 
                encoder's final hidden representation.
            encoder_output: The output of the encoder with shape
                (max_src_sequence_length, batch_size, hidden_size).
            encoder_mask: The output mask from the encoder with shape
                (max_src_sequence_length, batch_size). Encoder outputs at positions
                with a True value correspond to padding tokens and should be ignored.

        Returns:
            A tuple with three elements:
                logits: A tensor with shape (batch_size,
                    vocab_size) containing unnormalized scores for the next-word
                    predictions at each position.
                decoder_hidden: tensor h_n with the same shape as last_hidden 
                    representing the updated decoder state after processing the 
                    decoder input.
                attention_weights: This will be implemented later in the attention
                    model, but in order to maintain compatible type signatures, we also
                    include it here. This can be None or any other placeholder value.
        """

        # These arguments are not used in the baseline model.
        del encoder_output
        del encoder_mask
        decoder_input = torch.unsqueeze(decoder_input, 0)

        x = sent.encode(vocab.id_to_word[decoder_input[0][0].item()])
        x = torch.Tensor(x).unsqueeze(0).cuda()
        x = x.unsqueeze(1)
 
        x, hid = self.decoder(x, last_hidden)

        x = self.lin(torch.squeeze(x, 0))


        return x, hid, 0
        # YOUR CODE HERE
        ...

    def compute_loss(self, source, target):
        """Run the model on the source and compute the loss on the target.

        Args:
            source: An integer tensor with shape (max_source_sequence_length,
                batch_size) containing subword indices for the source sentences.
            target: An integer tensor with shape (max_target_sequence_length,
                batch_size) containing subword indices for the target sentences.

        Returns:
            A scalar float tensor representing cross-entropy loss on the current batch
            divided by the number of target tokens in the batch.
            Many of the target tokens will be pad tokens. You should mask the loss 
            from these tokens using appropriate mask on the target tokens loss.
        """

        # Implementation tip: don't feed the target tensor directly to the decoder.
        # To see why, note that for a target sequence like <s> A B C </s>, you would
        # want to run the decoder on the prefix <s> A B C and have it predict the
        # suffix A B C </s>.
        x, mask, hid = self.encode(source)

        loss_func = nn.CrossEntropyLoss()
        loss = 0
        for i in range(target.shape[0]- 1):
          out, hid, temp = self.decode(target[i], hid, x, mask)

          loss += loss_func(out, target[i + 1])
        return loss/target.shape[0]

        # You may run self.encode() on the source only once and decode the target 
        # one step at a time.

        # YOUR CODE HERE
        ...

In [105]:
def train(model, train, label, num_epochs, model_file, learning_rate=0.0001):
    """Train the model for given number of epochs and save the trained model in 
    the final model_file.
    """

    decoder_learning_ratio = 5.0
    
    encoder_parameter_names = ["E", "encoder"]
                               
    encoder_named_params = list(filter(lambda kv: any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    decoder_named_params = list(filter(lambda kv: not any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    encoder_params = [e[1] for e in encoder_named_params]
    decoder_params = [e[1] for e in decoder_named_params]
    optimizer = torch.optim.AdamW([{'params': encoder_params},
                {'params': decoder_params, 'lr': learning_rate * decoder_learning_ratio}], lr=learning_rate)
    
    clip = 50.0
    for epoch in range(num_epochs):
        print(epoch)
        # print(f"Total training instances = {len(train_dataset)}")
        # print(f"train_data_loader = {len(train_data_loader)} {1180 > len(train_data_loader)/20}")
        model.train()
        total_loss = 0.0
        for i in range(len(train)):
          if i == len(train)/2:
            print("half_way")
          source = torch.Tensor(train[i]).unsqueeze(1).cuda()
          tgt_ids = vocab.get_ids_from_sentence(label[i])
          target = torch.LongTensor(tgt_ids).unsqueeze(1).cuda()
          optimizer.zero_grad()
          loss = model.compute_loss(source, target)
          total_loss += loss.item()
          loss.backward()
          # Gradient clipping before taking the step
          _ = nn.utils.clip_grad_norm_(model.parameters(), clip)
          optimizer.step()

    # Save the model after training         


In [106]:
# You are welcome to adjust these parameters based on your model implementation.
num_epochs = 10

baseline_model = Seq2seqBaseline(vocab).cuda()
train(baseline_model, sent_emb, labels, num_epochs, "sent_emb_model.pt")
# Download the trained model to local for future use
#files.download('baseline_model.pt')

0
half_way
1
half_way
2
half_way
3
half_way
4
half_way
5
half_way
6
half_way
7
half_way
8
half_way
9
half_way


In [107]:
torch.save(baseline_model.state_dict(), "bert_sent_seq2seq.pt")

In [108]:
def predict_greedy(model, sentence, max_length=100):
    """Make predictions for the given input using greedy inference.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: A input string.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        Model's predicted greedy response for the input, represented as string.
    """

    # You should make only one call to model.encode() at the start of the function, 
    # and make only one call to model.decode() per inference step.
    model.eval()
    source = torch.unsqueeze(torch.tensor(sentence).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    sent = [start]
    i = 0
    while start != eos_id and i < 100:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
          start = torch.argmax(out[0], 0)
          sent.append(start.item())
          i += 1
    sent = vocab.decode_sentence_from_ids(sent)
    
    return sent
score = 0
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
for i in range(0, len(sent_emb), 10):
  if jaccard(predict_greedy(baseline_model, sent_emb[i]), labels[i])>= 0.5:
    score += 1
print(score*10/len(sent_emb))


0.7255845942228336


In [113]:
def predict_beam(model, sentence, k=3, max_length=100, thresh=-9999):
    """Make predictions for the given inputs using beam search.
    
    Args:
        model: A sequence-to-sequence model.
        sentence: An input sentence, represented as string.
        k: The size of the beam.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.
    
    Returns:
        A list of k beam predictions. Each element in the list should be a string
        corresponding to one of the top k predictions for the corresponding input,
        sorted in descending order by its final score.
    """

    # Implementation tip: once an eos_token has been generated for any beam, 
    # remove its subsequent predictions from that beam by adding a small negative 
    # number like -1e9 to the appropriate logits. This will ensure that the 
    # candidates are removed from the beam, as its probability will be very close
    # to 0. Using this method, uou will be able to reuse the beam of an already 
    # finished candidate

    # Implementation tip: while you are encouraged to keep your tensor dimensions
    # constant for simplicity (aside from the sequence length), some special care
    # will need to be taken on the first iteration to ensure that your beam
    # doesn't fill up with k identical copies of the same candidate.
    
    # You are welcome to tweak alpha
    alpha = 0.9
    model.eval()
    beams = []
    curr = []
    source = torch.unsqueeze(torch.tensor(sentence).cuda(), 1)
    x, mask, hid = model.encode(source)
    start = bos_id
    
    out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(start).cuda(), 0), hid, x, mask)
    out = torch.log_softmax(out[0], 0)
    values, start = torch.topk(out, k, 0)
    for i in range(len(values)):
      # Each beam contains the log probs at its first index and the hidden states at its last index
      beams.append([values[i], start[i].item(), hid])
    
    generation = []
    i = 0
    while i < k:
      curr = []
      for j in beams:
          out, hid, temp = model.decode(torch.unsqueeze(torch.tensor(j[-2]).cuda(), 0), j[-1], x, mask)
          out = torch.log_softmax(out[0], 0)
          values, start = torch.topk(out, k, 0)
          for z in range(len(values)):
            temp = j.copy()
            temp[0] = values[z] + temp[0]
            temp.insert(-1, start[z].item())
            temp[-1] = hid
            curr.append(temp)
      curr = sorted(curr,reverse=True, key=lambda x: x[0])
      curr = curr[0:k - i]
      beams = []
      for j in curr:
        if j[-2] == eos_id or len(j) > 20:
          generation.append(j[:-1])
          i +=1
        else:
          beams.append(j)
    final = []
    generation = sorted(generation, reverse=True, key=lambda x: x[0]/(len(x)-1)**alpha)
    for i in generation:
      if i[0].item() > thresh:
        final.append(vocab.decode_sentence_from_ids(i[1:]))
    return final
score = 0
for i in range(0, len(sent_emb), 10):
  predictions = predict_beam(baseline_model, sent_emb[i], thresh=-2)
  for j in predictions:
    if jaccard(j, labels[i])>= 0.5:
      score += 1
      break
print(score*10/len(sent_emb))


0.859697386519945


In [147]:
testing = {}
for i in range(0, len(sent_emb)):
  if sent_emb[i][0][0] not in testing.keys():
    predictions = predict_beam(baseline_model, sent_emb[i], thresh=-3)
    testing[sent_emb[i][0][0]] = (predictions, [labels[i]])
  else:
    testing[sent_emb[i][0][0]][1].append(labels[i])

In [150]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  cop = predictions.copy()
  true_pred = i[1].copy()
  check = False
  #check exact match first
  for j in predictions:
    if j in true_pred:
      tp += 1
      true_pred.remove(j)
      cop.remove(j)
  #then check rest for jaccard score
  for j in cop:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [151]:
print("training performance")
print("micro F score")
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))
print("accuracy")
print(tp/(tp+fn))

training performance
micro F score
1000
481
0.7662194159431729
accuracy
0.8345942228335625


In [158]:
testing = {}
for i in range(0, len(test_x)):
  if test_x[i][0][0] not in testing.keys():
    predictions = predict_beam(baseline_model, test_x[i], thresh=-3)
    testing[test_x[i][0][0]] = (predictions, [test_y[i]])
  else:
    testing[test_x[i][0][0]][1].append(test_y[i])

In [159]:
tp = 0
fp = 0
fn = 0
for i in testing.values():
  predictions = i[0]
  cop = predictions.copy()
  true_pred = i[1].copy()
  check = False
  #check exact match first
  for j in predictions:
    if j in true_pred:
      tp += 1
      true_pred.remove(j)
      cop.remove(j)
  #then check rest for jaccard score
  for j in cop:
    found = False
    removal = 0
    for k in true_pred:
      if jaccard(j, k) >= 0.5:
        found = True
        removal = k
        break
    if found:
      tp += 1
      true_pred.remove(removal)
    else:
      fp += 1
  fn += len(true_pred)

In [161]:
print("testing performance")
print("micro F score")
print(fp)
print(fn)
print(tp/(tp + 1/2*(fp+fn)))
print("accuracy")
print(tp/(tp+fn))

testing performance
micro F score
863
255
0.6395873629916183
accuracy
0.7955092221331195
